In [5]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import itertools

from collections import defaultdict

In [6]:
import nltk
import random
from nltk.classify.scikitlearn import SklearnClassifier
import pickle
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from nltk.classify import ClassifierI
from statistics import mode
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from nltk.stem import PorterStemmer

In [6]:
#import all the necessary csv files
parent_dir = 'D:/DCU Documents/Semester 2/Data Analytics and Data Mining/Project/Kaggle Dataset/'

path_dic = {
            'B':parent_dir+'business_s.csv',
            'R':parent_dir+'Review2021.csv',
            'U':parent_dir+'U.csv',
            'D':parent_dir+'documents.csv'
           }

documents = pd.read_csv(path_dic['D'])

In [7]:
documents.columns
documents.drop(documents.columns[[0]], axis = 1, inplace = True)

In [8]:
documents = documents. to_records(index=False)
documents = list(documents)
#print(result)

In [9]:
ps = PorterStemmer()
all_words = ""
Stemmed_data = []
for (t, star) in documents:
    for word in t.split():
        w = word.lower().replace('.', '').replace(',', '').replace('!', '')
        all_words = all_words + ps.stem(w) +" "
    Stemmed_data.append((all_words,star))
    all_words = ""
        
#all_words = nltk.FreqDist(all_words)
#print(all_words.most_common(15))
#print("stupid appeared: " + str(all_words['stupid']) + " times")

In [10]:
Stemmed_data[2]

("sorri but it wa not good we order chicken marsala d- it wa suppos to be creami it wasn't no prosciutto dri chicken gyro f+ meat wa dri sauc on the side pita pizza c too much sauc but the babi like it burger b+ but you can get a burger anywher the place had a cigarett or mold type of smell but i do have to say in the begin we felt kind of rush but our second waitress if you i had to go back and trust me i won't it will have to be for her servic ",
 2.0)

In [11]:
documents[0:3]

[("They can't manage to get online orders right and no one there answers the phone to be able to correct them. They don't even have a voicemail to leave a message.", 1.),
 ("The hamburger and fries were surprisingly delicious.  Great value for the money.  My partner had the burger and onion rings.  The rings were way above expectations.   I'll get the next time I'm here.", 4.),
 ("Sorry but it was not good we order chicken Marsala D- it was supposed to be creamy it wasn't \nNo prosciutto dry chicken \nGyro F+ meat was dry sauce on the side \nPita pizza c too much sauce but the baby like it\nBurger B+ but you can get a burger anywhere \nThe place had a cigarette or mold type of smell \nBut I do have to say in the beginning we felt kind of Rush but our second waitress if you I had to go back and trust me I won't it will have to be for her service!!!!", 2.)]

In [12]:
df = pd.DataFrame(Stemmed_data, columns=['text_stem', 'star'])

In [26]:
df["pos"][8]

0.0

In [14]:
df = df[df["star"] != 3.0]

In [15]:
df.loc[df.star >= 4, 'pos'] = 1
df.loc[df.star <= 2, 'pos'] = 0

In [16]:
X_train, X_test, y_train, y_test = train_test_split((df['text_stem']), (df['pos']), train_size = .75, random_state = 47)
print("Shape of X_train: ", X_train.shape)
print("Shape of X_test: ", X_test.shape)

Shape of X_train:  (94367,)
Shape of X_test:  (31456,)


# Count Vectorizer

In [27]:
vect = CountVectorizer(max_df = 0.95, min_df = 3, stop_words = 'english').fit(X_train)
vect.get_feature_names()[::2000]

['00',
 'bacchanalia',
 'chewabl',
 'disinfect',
 'funfetti',
 'instructor',
 'marriott',
 'oysters',
 'religi',
 'soltani',
 'true']

In [28]:
print("Total number of features: ", len(vect.get_feature_names()))

Total number of features:  21857


In [29]:
# Now we vectorize the X_train data
X_train_vectorized = vect.transform(X_train)
X_train_vectorized

<94367x21857 sparse matrix of type '<class 'numpy.int64'>'
	with 3515648 stored elements in Compressed Sparse Row format>

In [31]:
clf = LinearSVC(max_iter=1200000)
clf.fit(X_train_vectorized, y_train)

pred = clf.predict(vect.transform(X_test))

print(confusion_matrix(y_true = y_test, y_pred = pred))
print("Test accuracy: ", (confusion_matrix(y_true = y_test, y_pred = pred)[0][0] + confusion_matrix(y_true = y_test, y_pred = pred)[1][1])/len(X_test))
print('AUC: ', roc_auc_score(y_test, pred))

[[ 7585   938]
 [  810 22123]]
Test accuracy:  0.9444303153611394
AUC:  0.9273122871399675


In [32]:
feature_names = np.array(vect.get_feature_names())

sored_coef_index = clf.coef_[0].argsort()
print("Smallest Coefs: \n{}\n".format(feature_names[sored_coef_index[:15]]))
print("Biggest Coefs: \n{}\n".format(feature_names[sored_coef_index[:-16:-1]]))

Smallest Coefs: 
['mislead' 'quincy' '1909' 'emul' 'caller' 'banner' 'pathet' 'basics'
 'brunt' 'unbias' 'likes' 'tediou' 'rudest' 'tractor' 'paddlefish']

Biggest Coefs: 
['sammy' 'daddi' 'appoin' 'murphy' 'magnific' 'panhandl' 'ingredients'
 'dynasti' 'graciou' 'phenomen' 'sprint' 'seconds' 'avo' 'started' 'caraf']



# TF-IDF

Tf–idf, or Term frequency-inverse document frequency, allows us to weight terms based on how important they are to a document. High weight is given to terms that appear often in a particular document, but don't appear often in the corpus. Features with low tf–idf are either commonly used across all documents or rarely used and only occur in long documents. 

In [33]:
vect = TfidfVectorizer(max_df = .95, min_df = 3, stop_words = 'english').fit(X_train)
len(vect.get_feature_names())

21857

In [34]:
X_train_vectorized = vect.transform(X_train)

clf = LinearSVC()
clf.fit(X_train_vectorized, y_train)

pred = clf.predict(vect.transform(X_test))

print(confusion_matrix(y_true = y_test, y_pred = pred))
print("Test accuracy: ", (confusion_matrix(y_true = y_test, y_pred = pred)[0][0] + confusion_matrix(y_true = y_test, y_pred = pred)[1][1])/len(X_test))
print('AUC: ', roc_auc_score(y_test, pred))

[[ 7739   784]
 [  604 22329]]
Test accuracy:  0.9558748728382502
AUC:  0.9408380090574874


In [35]:
feature_names = np.array(vect.get_feature_names())

sored_coef_index = clf.coef_[0].argsort()
print("Smallest Coefs: \n{}\n".format(feature_names[sored_coef_index[:25]]))
print("Biggest Coefs: \n{}\n".format(feature_names[sored_coef_index[:-26:-1]]))

Smallest Coefs: 
['worst' 'bland' 'terribl' 'horribl' 'rude' 'disgust' 'unaccept' 'mediocr'
 'unprofession' 'aw' 'tasteless' 'poor' 'mislead' 'scam' 'wors'
 'flavorless' 'downhil' 'unfortun' 'underwhelm' 'zero' 'bare' 'subpar'
 'lack' 'insult' 'stale']

Biggest Coefs: 
['delici' 'amaz' 'great' 'excel' 'perfect' 'best' 'love' 'outstand'
 'perfectli' 'phenomen' 'thank' 'fantast' 'awesom' 'highli' 'appreci'
 'glad' 'friendli' 'yummi' 'tasti' 'good' 'thorough' 'favorit'
 'profession' 'gem' 'grate']



In [36]:
# The issue with n-grams: do not, not recommend, not good
print(clf.predict(vect.transform(['do not recommend this place',
                                 'this place is not good'])))

[1. 1.]


# n-grams

In [37]:
vect = TfidfVectorizer(max_df = .95, min_df = 3, ngram_range = (1,2)).fit(X_train)
print(len(vect.get_feature_names()))

293979


In [38]:
X_train_vectorized = vect.transform(X_train)

clf = LinearSVC()
clf.fit(X_train_vectorized, y_train)

pred = clf.predict(vect.transform(X_test))

print(confusion_matrix(y_true = y_test, y_pred = pred))
print("Test accuracy: ", (confusion_matrix(y_true = y_test, y_pred = pred)[0][0] + confusion_matrix(y_true = y_test, y_pred = pred)[1][1])/len(X_test))
print('AUC: ', roc_auc_score(y_test, pred))

[[ 8042   481]
 [  399 22534]]
Test accuracy:  0.9720244150559512
AUC:  0.9630829819521445


In [39]:
feature_names = np.array(vect.get_feature_names())

sored_coef_index = clf.coef_[0].argsort()
print("Smallest Coefs: \n{}\n".format(feature_names[sored_coef_index[:45]]))
print("Biggest Coefs: \n{}\n".format(feature_names[sored_coef_index[:-46:-1]]))

Smallest Coefs: 
['not' 'rude' 'worst' 'terribl' 'veri disappoint' 'horribl' 'bland' 'no'
 'disappoint' 'poor' 'disgust' 'not worth' 'overpr' 'not good' 'mediocr'
 'aw' 'unfortun' 'not recommend' 'bare' 'at best' 'wors' 'unprofession'
 'lack' 'bad' 'two star' 'the worst' 'told' 'elsewher' 'never' 'money'
 'dri' 'zero' 'do not' 'don' 'won' 'shame' 'cold' 'unaccept' 'trash'
 'scam' 'will not' 'ridicul' 'so disappoint' 'wouldn recommend' 'subpar']

Biggest Coefs: 
['great' 'amaz' 'delici' 'love' 'best' 'excel' 'not disappoint' 'good'
 'thank' 'perfect' 'be disappoint' 'awesom' 'never disappoint' 'fantast'
 'definit' 'appreci' 'friendli' 'tasti' 'the best' 'outstand' 'nice'
 'perfectli' 'highli recommend' 'happi' 'beauti' 'profession' 'glad'
 'recommend' 'favorit' 'comfort' 'thank you' 'effici' 'alway' 'easi'
 'so good' 'enjoy' 'not bad' 'highli' 'will definit' 'yummi' 'fresh'
 'help' 'wonder' 'and' 'wa abl']



In [40]:
pos_words = pd.DataFrame(feature_names[sored_coef_index[:200]], columns = ['positive_words'])
pos_words.to_csv(parent_dir+'Positive Words.csv')
neg_words = pd.DataFrame(feature_names[sored_coef_index[:-200:-1]], columns = ['negative_words'])
neg_words.to_csv(parent_dir+'Negative Words.csv')

In [41]:
# NO MORE issue with n-grams: do not, not recommend, not good
print(clf.predict(vect.transform(['do not recommend this place',
                                 'this place is not good'])))

[0. 0.]


## Trying the voting clf

In [43]:
np.random.seed(47)
# LogisticRegression
clf_LogisticRegression = LogisticRegression(max_iter=1200000,class_weight = 'balanced') #class_weight = 'balanced'
clf_LogisticRegression.fit(X_train_vectorized, y_train)

pred = clf_LogisticRegression.predict(vect.transform(X_test))
print("LogisticRegression accuracy: ", (confusion_matrix(y_true = y_test, y_pred = pred)[0][0] + confusion_matrix(y_true = y_test, y_pred = pred)[1][1])/len(X_test))
print(confusion_matrix(y_true = y_test, y_pred = pred))
print('AUC: ', roc_auc_score(y_test, pred))

# LinearSVC
clf_LinearSVC = LinearSVC() #class_weight = 'balanced'
clf_LinearSVC.fit(X_train_vectorized, y_train)

pred = clf_LinearSVC.predict(vect.transform(X_test))
print("LinearSVC accuracy: ", (confusion_matrix(y_true = y_test, y_pred = pred)[0][0] + confusion_matrix(y_true = y_test, y_pred = pred)[1][1])/len(X_test))
print(confusion_matrix(y_true = y_test, y_pred = pred))
print('AUC: ', roc_auc_score(y_test, pred))

# SGDClassifier
clf_SGDClassifier = SGDClassifier(class_weight = 'balanced') #class_weight = 'balanced'
clf_SGDClassifier.fit(X_train_vectorized, y_train)

pred = clf_SGDClassifier.predict(vect.transform(X_test))
print("SGDClassifier accuracy: ", (confusion_matrix(y_true = y_test, y_pred = pred)[0][0] + confusion_matrix(y_true = y_test, y_pred = pred)[1][1])/len(X_test))
print(confusion_matrix(y_true = y_test, y_pred = pred))
print('AUC: ', roc_auc_score(y_test, pred))

LogisticRegression accuracy:  0.9640132248219736
[[ 8223   300]
 [  832 22101]]
AUC:  0.9642607646383946
LinearSVC accuracy:  0.9720244150559512
[[ 8042   481]
 [  399 22534]]
AUC:  0.9630829819521445
SGDClassifier accuracy:  0.958736012207528
[[ 8263   260]
 [ 1038 21895]]
AUC:  0.9621160118631955


Trying out the voted clf

In [44]:
class VoteClassifier(ClassifierI):
    def __init__(self, *classifiers):
        self._classifiers = classifiers

    def predict(self, features):
        pred_list = []
        for c in self._classifiers:
            pred_list.append(c.predict(features))
        
        res = []
        for i in range(len(pred_list[0])):
            a = pred_list[0][i]
            b = pred_list[1][i]
            c = pred_list[2][i]
            if (a+b+c)< 2:
                res.append(0)
            else:
                res.append(1)
        return res

    def confidence(self, features):
        pred_list = []
        for c in self._classifiers:
            pred_list.append(c.predict(features))
        
        votes = pred_list[0][0] + pred_list[1][0] + pred_list[2][0]
        if votes< 2:
            return 1-votes/3
        else:
            return votes/3
    
voted_classifier = VoteClassifier(clf_LogisticRegression,
                                  clf_LinearSVC, 
                                  clf_SGDClassifier) 

pred = voted_classifier.predict(vect.transform(X_test))
print("voted_classifier accuracy: ", (confusion_matrix(y_true = y_test, y_pred = pred)[0][0] + confusion_matrix(y_true = y_test, y_pred = pred)[1][1])/len(X_test))

voted_classifier accuracy:  0.9641721770091557


In [45]:
voted_classifier.predict(vect.transform(['do not recommend this place',
                                 'this place is not good']))

[0, 0]

In [46]:
def sentiment(text):
    return (voted_classifier.predict(vect.transform(text))[0], voted_classifier.confidence(vect.transform(text)))

sentiment(['I do not recommend this place'])

(0, 1.0)

In [47]:
sentiment(["we will come back again."])


(1, 1.0)

In [8]:
def clean_format(value):
    return value.lower().replace('.', '').replace(',', '').replace('!', '')

In [11]:
ps = PorterStemmer() 
example_sec = 'You have to build a very good site and I love visiting your site.'

from nltk.corpus import stopwords
stopWords = list(set(stopwords.words('english')))

print([w for w in word_tokenize(example_sec)])
print([clean_format(w) for w in word_tokenize(example_sec)])
print([clean_format(w) for w in word_tokenize(example_sec) if w not in stopWords])
print([ps.stem(clean_format(w)) for w in word_tokenize(example_sec) if w not in stopWords])

['You', 'have', 'to', 'build', 'a', 'very', 'good', 'site', 'and', 'I', 'love', 'visiting', 'your', 'site', '.']
['you', 'have', 'to', 'build', 'a', 'very', 'good', 'site', 'and', 'i', 'love', 'visiting', 'your', 'site', '']
['you', 'build', 'good', 'site', 'i', 'love', 'visiting', 'site', '']
['you', 'build', 'good', 'site', 'i', 'love', 'visit', 'site', '']


In [51]:
sens = ["The python", "The python programmer named pythoner ", "The python programmer named pythoner is pythoning a game pythonly"]
samp_list = np.array([" ".join([ps.stem(clean_format(w)) for w in word_tokenize(example_sec) if w not in stopWords]) for example_sec in sens])
# samp_list
transed = vect.transform(samp_list)

In [52]:
ps.stem('amazing')

'amaz'

In [54]:
df.to_csv(parent_dir+"document_pos_value.csv")